## First Look at Gravity Data

#### Import Data Files

In [ ]:
%reset -f

In [ ]:
import pandas as pd
import hvplot.pandas
import numpy as np
import matplotlib.dates as dates
import warnings
warnings.filterwarnings('ignore')
import holoviews as hv
from holoviews import dim, opts
import hvplot.dask
hv.extension('bokeh')
import glob, os
import dask.dataframe as dd

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

In [ ]:
import datetime
def dateparse (date_string):
    return datetime.datetime.strptime(date_string, '%d-%m-%Y %H:%M:%S')

def dateparseSPAIN (date_string):
    return datetime.datetime.strptime(date_string, '%Y-%m-%d-%H:%M:%S')

In [ ]:
#!head ~jovyan/data/bravoseis/gravity/gravimetro_bruto/21012019.gravimetro_bruto.proc

In [ ]:
!ls /home/jovyan/data/bravoseis_data/SADO/jan_2019/gravimetro_bruto.proc/

In [ ]:
#!head ~/Dropbox/QueensCollege/Research/BransfieldStrait/data/grav/raw/21012019.gravimetro_bruto.raw

## Read Gravity Files

In [ ]:
%%time
path = '/home/jovyan/data/bravoseis_data/SADO/jan_2019/gravimetro_bruto.proc/' # use your path

all_files = glob.glob(os.path.join(path, "*.proc"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f, parse_dates=True, date_parser=dateparse, index_col='fecha',
                       dtype = {'Date': object,'status': np.float64,
                                'gravimetria_bruta': np.float64, 'spring_tension': np.float64,
                                'longitud': np.float64, 'latitud': np.float64,
                                'velocidad': np.float64,'rumbo': np.float64 }) for f in all_files)

concatenated_df   = pd.concat(df_from_each_file, ignore_index=False)
df_grav   = concatenated_df.sort_values(by='fecha_telegrama')
df_grav['date']=df_grav.index.date
df_grav['time']=df_grav.index.time
df_grav['survey']= np.nan # Survey Part
df_grav['loca'] = np.nan # Survey Location 
df_grav['line']= np.nan # Line Number 

In [ ]:
df_grav.head()

## Read Bathy Files

In [ ]:
!head /home/jovyan/data/bravoseis_data/SADO/jan_2019/posicion.proc/01012019.posicion.proc

In [ ]:
%%time
path = '/home/jovyan/data/bravoseis_data/SADO/jan_2019/posicion.proc/' # use your path

all_files = glob.glob(os.path.join(path, "*.proc"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_Bath = (pd.read_csv(f, parse_dates=True, date_parser=dateparse, index_col='fecha',
                       dtype = {'Date': object,'longitud': np.float64,
                                'latitud': np.float64, 'rumbo': np.float64,
                                'velocidad': np.float64, 'profundidad': np.float64,
                                'cog': np.float64,'sog': np.float64 }) for f in all_files)

concatBathy_df   = pd.concat(df_from_each_Bath, ignore_index=False)
df_bath   = concatBathy_df.sort_values(by='fecha_telegrama')

In [ ]:
df_bath.head()

In [ ]:
#df_bath.tail()

In [ ]:
!head /home/jovyan/data/bravoseis_data/MAGNETOMETRO/190121_0001.mag

## Read Mag Files

In [ ]:
%%time
path = '/home/jovyan/data/bravoseis_data/MAGNETOMETRO/' # use your path

all_files = glob.glob(os.path.join(path, "*.proc"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f, parse_dates=True, date_parser=dateparse, index_col='fecha',
                       dtype = {'Date': object,'status': np.float64,
                                'gravimetria_bruta': np.float64, 'spring_tension': np.float64,
                                'longitud': np.float64, 'latitud': np.float64,
                                'velocidad': np.float64,'rumbo': np.float64 }) for f in all_files)

concatenated_df   = pd.concat(df_from_each_file, ignore_index=False)
df_grav   = concatenated_df.sort_values(by='fecha_telegrama')
df_grav['date']=df_grav.index.date
df_grav['time']=df_grav.index.time
df_grav['survey']= np.nan # Survey Part
df_grav['loca'] = np.nan # Survey Location 
df_grav['line']= np.nan # Line Number 

In [ ]:
df_mag.head()

### Merge Dataframes

In [ ]:
test = pd.merge(df_bath, df_grav,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_gravMerge = pd.DataFrame()
df_gravMerge = test[test['_merge'] == 'both']
del df_gravMerge['_merge']
df_gravMerge['longitud'] = (df_gravMerge['longitud_B'] + df_gravMerge['longitud_G'])/2
df_gravMerge['latitud'] = (df_gravMerge['latitud_B'] + df_gravMerge['latitud_G'])/2
df_gravMerge['rumbo'] = (df_gravMerge['rumbo_B'] + df_gravMerge['rumbo_G'])/2
df_gravMerge['velocidad'] = (df_gravMerge['velocidad_B'] + df_gravMerge['velocidad_G'])/2
del df_gravMerge['longitud_B']
del df_gravMerge['latitud_B']
del df_gravMerge['rumbo_B']
del df_gravMerge['velocidad_B']
del df_gravMerge['fecha_telegrama_B']
del df_gravMerge['longitud_G']
del df_gravMerge['latitud_G']
del df_gravMerge['rumbo_G']
del df_gravMerge['velocidad_G']
del df_gravMerge['fecha_telegrama_G']
del df_gravMerge['status']
df_gravMerge.head()

In [ ]:
!tail bravoseis_tables.csv

In [ ]:
df_lineNumbers = (pd.read_csv('bravoseis_tables.csv', parse_dates=[3, 4], date_parser=dateparseSPAIN))
df_lineNumbers.columns = ['survey','loca','line',
                     's_time','e_time']
#df_lineNumbers.head(50)

In [ ]:
df_lineNumbers.dtypes

In [ ]:
pd.to_datetime(df_lineNumbers.e_time, format = "%Y-%m-%d-%H:%M:%S");

In [ ]:
for index, row in df_lineNumbers.iterrows():
    mask = (df_gravMerge.index > row.s_time) & (df_gravMerge.index <= row.e_time)
    df_gravMerge.survey[mask]= row.survey
    df_gravMerge.loca[mask]= row.loca
    df_gravMerge.line[mask]= row.line

In [ ]:
gravimetria = df_gravMerge.dropna()
gravimetria.head()

In [ ]:
gravimetria = gravimetria[gravimetria.loca != 'EdifaceA'];
gravimetria = gravimetria[gravimetria.loca != 'Transit'];
gravimetria = gravimetria[gravimetria.line != 'RIF12'];
gravimetria = gravimetria[gravimetria.line != 'RIF11'];
gravimetria = gravimetria[gravimetria.line != 'RIF11b'];
gravimetria = gravimetria[gravimetria.line != 'RIF10'];
gravimetria = gravimetria[gravimetria.line != 'RIF04'];
gravimetria = gravimetria[gravimetria.line != 'RIF03'];
gravimetria = gravimetria[gravimetria.line != 'RIF02'];
gravimetria = gravimetria[gravimetria.line != 'ORK10'];
gravimetria = gravimetria[gravimetria.line != 'ORK17'];
gravimetria = gravimetria[gravimetria.line != 'ORK02']; #Start of line for OR_2 at 08:59 UTC. The streamer is completely outside the line (Feather angle of -13 º in the good part of the line). At 1300 UTC end of line for OR_2
gravimetria = gravimetria[gravimetria.line != 'ORK05'];
gravimetria = gravimetria[gravimetria.line != 'ORK18b'];#We begin the Turn B with the line OR18. There is a large iceberg 4 km away, at the moment we will not deviate from the line. At 15:10 (UTC) the guns have tangled with the streamer. They stopped the acquisition of data. At 18:05 we returned to the initial point to redo the survey of the line 2300 UTC bad weather, large waves make multibeam data poor quality. 
gravimetria = gravimetria[gravimetria.line != 'T10'];


gravimetria = gravimetria.rename(columns={"profundidad": "Depth", "longitud": "Longitude", "latitud": "Latitude"});

In [ ]:
gravimetria.head()

### Latitude Corrections 
g(phi) = 978.0327(1+0.0052792 sin^2(phi) - 0.0000232sin^4(phi)

### Free-air
* F1 = F2 = G (m1m2/r^2)
* G = 6.670 x 10 N-m^2/kg^2
* r = distance between point masses 
* F1, F2 = force of attraction 
* F= 0.3086 mGal/m

### B(rho) = 2*pi*G*rho_Bouguer*h
* rho_Bouguer = Bouguer Density in kg/m^3
* h = elevation above sea level in meters
#### A 2.67 g/cm^3 Bouguer density is commonly used as a good average density for continental crust. Given a Bouguer density, we 

### E

In [ ]:
#gravimetria.to_csv('gravimetria.csv');

In [ ]:
#gravimetria.sample(100)

In [ ]:
gravimetria.hvplot.points('fecha', 'gravimetria_bruta', color='gravimetria_bruta',
                             cmap='colorwheel', size=.5,
                             hover_cols=['cog', 'line'], title= 'proc_gravity')

In [ ]:
gravimetria.hvplot.scatter('Longitude', 'Latitude', 
                      height=500, 
                      color ='gravimetria_bruta', 
                      cmap='colorwheel', 
                      size=50, 
                      hover_cols=['line'], title= 'proc_gravity subset')

### Downsample the data

In [ ]:
df_minuteGrav = pd.DataFrame()
df_minuteGrav['gravimetria_bruta'] = gravimetria.gravimetria_bruta.resample('min').mean()
df_minuteGrav['lon'] = gravimetria.Longitude.resample('min').mean()
df_minuteGrav['lat'] = gravimetria.Latitude.resample('min').mean()
df_minuteGrav['depth'] = gravimetria.Depth.resample('min').mean()
df_minuteGrav.tail()

In [ ]:
df_minuteGrav2.hvplot.points('index', 'proc_gravity', color='proc_gravity',
                             cmap='colorwheel', size=.5,
                             hover_cols=['cog'], title= 'proc_gravity')

In [ ]:
df_minuteGrav.hvplot.scatter('lon ', 'lat', 
                      height=500, 
                      color ='gravimetria_bruta', 
                      cmap='colorwheel', 
                      size=50, 
                      hover_cols=['depth'], title= 'proc_gravity subset')

#### Gravity is measured in MGals

In [ ]:
df_gravMerge['eotvos'] = 4.040 * df_gravMerge['sog'].values * df_gravMerge['cog'].apply(np.sin)* df_gravMerge['latitud'].apply(np.cos) + (0.001211 * df_gravMerge['sog']**2 )
df_gravMerge.head()

## Latitude Correction 
https://rallen.berkeley.edu/teaching/F04_GEO594_IntroAppGeophys/Lectures/L03_GravCorrAnalysis.pdf
#### Geodetic Reference System (GRS-1967) formula
#### gφ =9.780318(1+0.0053024sin2φ−0.0000059sin2 2φ) m/s2

## Bouguer correction

#### Accounts for rock thickness between current and base station elevation
#### Treat the rock as an infinite horizontal slab:
#### CB = 0.000419∆hρ where ∆h is in m and ρ is in km/m3

In [ ]:
#df_gravMerge.size

In [ ]:
df_minuteGrav.size

In [ ]:
df_minuteGrav2=df_minuteGrav.loc['2019-01-20 00:00:00':'2019-01-24 00:00:00']
df_temp=df_minuteGrav.loc['2019-01-26 21:00:00':'2019-02-05 23:58:00']
df_minuteGrav2=df_minuteGrav2.append(df_temp)

In [ ]:
df_minuteGrav2.hvplot.points('lon', 'lat', 
                      height=500, 
                      color='proc_gravity', 
                      cmap='colorwheel', 
                      size=3, 
                      hover_cols=['depth'], title= 'proc_gravity',
                      fontsize={'title': 16, 'labels': 14, 'xticks': 12, 'yticks': 12})

In [ ]:
#df_minuteGrav2.hvplot.heatmap(x='lon', y='lat', C='proc_gravity', reduce_function=np.mean, colorbar=True)

### Things to notice:
1. The depth signiature is visable
2. Examine crossing paths... there is a directioal dependence to our readings related to ship direction. 
3. Is the difference between these lines just the ETVOS correction or are their other corrections that need to be applied? 
4. Whould you please share the processing stream? 

In [ ]:
df_minuteGrav2.hvplot.points('index', 'proc_gravity', color='proc_gravity',
                             cmap='colorwheel', size=.5,
                             hover_cols=['cog'], title= 'proc_gravity')

In [ ]:
df_minuteGrav2.head(1)

In [ ]:
cond1 = df_minuteGrav2["lat"] < -62.44    
cond2 = df_minuteGrav2["lat"] > -62.45
cond3 = df_minuteGrav2["lon"] > -58.42
cond4 = df_minuteGrav2["lon"] < -58.36

df_minuteGrav3 = df_minuteGrav2[cond1 & cond2 & cond3 & cond4]

In [ ]:
del df_minuteGrav3['eotvos']
del df_minuteGrav3['grav_corr']
df_minuteGrav3.head()

In [ ]:
df_minuteGrav3.hvplot.scatter('lon', 'lat', 
                      height=500, 
                      color='proc_gravity', 
                      cmap='colorwheel', 
                      size=50, 
                      hover_cols=['depth'], title= 'proc_gravity subset').opts(bgcolor= grey)

In [ ]:
df_minuteGrav3.to_csv('proc_gravity_subset.csv') 

## The gravitational constant in SI units :math:`m^3 kg^{-1} s^{-1}`
## GRAVITATIONAL_CONST = 0.00000000006673


#### If terrain corrections (see below) are not applied, the term simple Bouguer anomaly is used. If they have, the term complete Bouguer anomaly is used. A second order correction to account for the curvature of the Earth is often added to this calculation.

In [ ]:
ellipsoid = get_ellipsoid()
 #Convert latitude to radians
    latitude_rad = np.radians(latitude)
     prime_vertical_radius = ellipsoid.semimajor_axis / np.sqrt(1 - ellipsoid.first_eccentricity ** 2 * np.sin(latitude_rad) ** 2)
        # Instead of computing X and Y, we only comupute the projection on the XY plane:
        # xy_projection = sqrt( X**2 + Y**2 )
 xy_projection = (height + prime_vertical_radius) * np.cos(latitude_rad)
 z_cartesian = (height + (1 - ellipsoid.first_eccentricity ** 2) * prime_vertical_radius) * np.sin(latitude_rad)
 radius = np.sqrt(xy_projection ** 2 + z_cartesian ** 2)
 geocentric_latitude = 180 / np.pi * np.arcsin(z_cartesian / radius)

    return geocentric_latitude, radius


#### Notes:
1. Seismic Velocites can be converted to density using the Carlson and Raskin [1984] velocity/ density relation for oceanic rocks that takes into account the porosity of layer 2. This relationshp can be used to approximate the gravitatational effects of the axial valley topography. 

2. Hooft 2000 suspended the density model from the bathymetry forllowing the method of Canales et al. [2000]

3. To avoid spatial aliasing, density interfaces were mirrored on all sides using the spectral method of Parker 1972

4. The crustal vravity signature and the mantle density variations due to the thermal structure of three-dimensional, plate driven mantle flow [Phipps Morgan and Forsyth, 1988] were subtracted from the observed free-air gravity anomaly to obtain the residual crustal Bouguer anomaly. 

5.  Marjanovic_2011 - Can we see evidence of Orca Volcano low mantle Bouguer gravity anomalies [Ito and Lin, 1995; Canales et al., 2002; Eysteinsson and Gunnarsson, 1995]?

6. Two‐ dimensional forward gravity modeling is conducted with the following primary goals: (1) to assess whether the axial density distributions inferred from gravity data are consistent with thicker crust beneath the Cleft and Endeavor segments, two ridge segments of proposed melt anomaly influ- ence, (2) to assess whether additional anomalous densities (i.e., in the mantle) are required to account for axial gravity anomalies, and (3) to evaluate constraints from gravity data on crustal structure of pseudofault zones and the origin of Moho travel time anomalies observed adjacent to pseudofaults.

[6] With gravity data, crustal thickness variations cannot be uniquely distinguished from variations in crust and/or mantle densities and the common approach is to evaluate a range of plausible models. With a few exceptions, constraints on crustal thickness from seismic studies are not typically available in prior gravity modeling studies of oce- anic crustal structure. Here, the available constraints from seismic data for the structure of uppermost crust (layer 2a) and for Moho reflection are used and a suite of models of varying middle‐to‐lower crustal structure are constructed. We investigate ridge axis structure using models of constant den- sity and thickness crust, constant density and var- iable thickness crust from the seismic reflection data, varying densities within the crust due to plate cooling away from the ridge axis, and varying densities within the mantle due to plate cooling.

Crustal structure at pseudofaults is investigated using best fit models of variable crustal densities given the seismic constraints on crustal thickness. The gravity models support the presence of thicker crust at both Cleft and Endeavor segments and require a broader zone of low densities in the underlying mantle beneath all segments. Preferred models for the ridge flank pseudofaults indicate local zones of thinner and thicker crust and higher densities. The crustal structure models are inter- preted in terms of implications for present‐day accretion processes along the JdF Ridge and at propagating ridge tips in the past.

As described previously, the primary goals of our study are to use gravity data to further inves- tigate anomalies in crustal structure inferred from seismic reflection data. Two‐dimensional forward gravity modeling along the three ridge flank pro- files is conducted. The GM‐SYS gravity/magnetic modeling software [Won and Bevis, 1987], pro- fessional basic version is used. The GM‐SYS package uses the method of Talwani et al. [1959] to calculate the gravitational attraction of two‐ dimensional bodies of arbitrary shape approxi- mated by an n‐sided polygon and constant density. All GM‐SYS models are extended to ±30,000 km (“infinity”) in the X direction to eliminate edge effects. Uniform structure perpendicular to the pro- file orientation is assumed with the 2‐D approxi- mation. While this assumption is well justified for the ridge axis and flanks where profile orientation is perpendicular to the dominant structural trends, it is less appropriate for the pseudofaults, which are oblique to the profile trend.

